# Instrument Classification - Part 1 : Pre-processing

We start our first trial by trying to recognise piano (pia), acoustic guitar (gac), saxophone (sax) and voice (voi).  

In [ ]:
import numpy as np
import librosa
import os
import math
from random import shuffle

In [ ]:
def split_train_test(dir_array):
    train_list = []
    test_list = []
    list = []
    for dir in dir_array:
        for f in os.listdir(dir):
            list.append(dir+f)
    shuffle(list)
    split_index = math.floor(len(list)*0.90)
    train_list = train_list + list[:split_index]
    test_list = test_list + list[split_index:]
    return (train_list,test_list)

def pre_processing(trainList,testList):
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    for y in [trainList,testList]:
        for f in y:
            label = f.split('/')
            wav,sr = librosa.load(f,mono=True)
            wav = wav / np.sqrt(np.mean(wav**2))
            mfcc = librosa.feature.melspectrogram(wav,n_mels =96,n_fft=1024,hop_length=256)
            if (y == trainList):
                y_train.append(label[4])
                x_train.append(mfcc)
            else:
                y_test.append(label[4])
                x_test.append(mfcc)
    return (x_train,y_train,x_test,y_test)

# [pia,gac,gel]
def to_categorical(list):
    temp_list = list
    for i in range(len(list)):
        if (list[i] == 'pia'):
            #temp_list[i] = [1,0,0,0]
            temp_list[i] = [1,0]
        elif (list[i] == 'gac'):
            temp_list[i] = [0,1]
        #elif (list[i] == 'sax'):
            #temp_list[i] = [0,0,1,0]
        #elif (list[i] == 'voi'):
            #temp_list[i] = [0,0,0,1]
        else:
            print("error")
    return temp_list

In [ ]:
# ~10 minutes to run
dirpia = './Desktop/Instrument-Classification/IRMAS-TrainingData/pia/'
dirgac = './Desktop/Instrument-Classification/IRMAS-TrainingData/gac/'
#dirsax = './Desktop/Instrument-Classification/IRMAS-TrainingData/sax/'
#dirvoi = './Desktop/Instrument-Classification/IRMAS-TrainingData/voi/'

(trainList,testList) = split_train_test([dirpia,dirgac])
(x_train,y_train,x_test,y_test) = pre_processing(trainList,testList)

y_train_new = np.array(to_categorical(y_train))
y_test_new = np.array(to_categorical(y_test))
x_train_new = np.expand_dims(np.array(x_train),-1)
x_test_new = np.expand_dims(np.array(x_test),-1)

print("Train Size: ", x_train_new.shape)
print("Test Size:  ", x_test_new.shape)

In [ ]:
np.save('x_train.npy',x_train_new)
np.save('x_test.npy',x_test_new)
np.save('y_train.npy',y_train_new)
np.save('y_test.npy',y_test_new)